# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
# For the purpose of simplification, we will not use string data
# on this Binary Classification project
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression

In [2]:
spark = SparkSession.builder.appName('logreg').getOrCreate()

In [3]:
# Get Data
df = spark.read.csv('customer_churn.csv',
                    header=True,inferSchema=True)

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [9]:
df.describe().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|summary|        Names|              Age|   Total_Purchase|   Account_Manager|            Years|         Num_Sites|       Onboard_date|            Location|             Company|              Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
|  count|          900|              900|              900|               900|              900|               900|                900|                 900|                 900|                900|
|   mean|         null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777|               null|                null|                null|0.16666666666666666|
| stddev| 

In [10]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [5]:
# BINARY CLASSIFICATION!

In [70]:
# Create the Vector Assembler to transform your data for ML
assembler = VectorAssembler(inputCols=['Age','Total_Purchase',
                                       'Years','Num_Sites'],
                            outputCol='features')

In [71]:
# Transform your data through the assembler
output = assembler.transform(df)

In [72]:
# Set up the final data for ML
finalData = output.select('features','Churn')

In [74]:
# Split the data
train_data,test_data = finalData.randomSplit([0.7,0.3])

In [76]:
# Initiate the model
logReg = LogisticRegression(labelCol='Churn')

In [77]:
# Fit the Model to train_data
fitModel = logReg.fit(train_data)

In [78]:
# View summary of train data
train_summary = fitModel.summary

In [81]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                638|                638|
|   mean|0.15987460815047022|0.11755485893416928|
| stddev| 0.3667772797614627|0.32233300248298724|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



In [82]:
# EVALUATE YOUR MODEL

In [83]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [84]:
pred_and_labels = fitModel.evaluate(test_data)

In [85]:
pred_and_labels.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|    0|[4.84819321579968...|[0.99221849221986...|       0.0|
|[25.0,9672.03,5.4...|    0|[4.46078437444934...|[0.98857865655897...|       0.0|
|[28.0,11128.95,5....|    0|[4.43011412519125...|[0.98822712182133...|       0.0|
|[29.0,5900.78,5.5...|    0|[4.37094777662312...|[0.98751850115704...|       0.0|
|[29.0,9617.59,5.4...|    0|[4.23948755704130...|[0.98578986200544...|       0.0|
|[29.0,12711.15,5....|    0|[5.14326438052284...|[0.99419529220059...|       0.0|
|[30.0,12788.37,4....|    0|[2.32250266369604...|[0.91072363175535...|       0.0|
|[31.0,7073.61,5.7...|    0|[2.92876082749769...|[0.94925001186573...|       0.0|
|[31.0,9574.89,7.3...|    0|[3.11047411333989...|[0.95732273031900...|       0.0|
|[31.0,11297.57,

In [87]:
# View AUC
churnEVal = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                          labelCol='Churn')

In [88]:
auc = churnEVal.evaluate(pred_and_labels.predictions)

In [89]:
auc # 74%

0.7406542056074766

In [90]:
# Now predict on new data, using the new_customers.csv
newData = spark.read.csv('new_customers.csv',
                         header=True,inferSchema=True)

In [92]:
newData.printSchema() # Same format except the label

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [91]:
# Fit the model to all the initial data you were given
finalLogModel = logReg.fit(finalData)

In [93]:
# Transform the new data for ML
test_newCustomers = assembler.transform(newData)

In [94]:
test_newCustomers.printSchema() # Check for features column

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- features: vector (nullable = true)



In [96]:
finalResults = finalLogModel.transform(test_newCustomers)

In [98]:
finalResults.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+



In [ ]:
# These are my predictions with roughly 74% accuracy.